<a href="https://colab.research.google.com/github/dimpy500/Google-Bitcoin-Analyst-project/blob/main/Google_Causal_Impact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries, Directories and Data

In [ ]:
%cd /content/drive/MyDrive/BUSINESS_ANALYST/Business Analyst course/Econometrics and Causal Inference/Google Causal Impact

/content/drive/MyDrive/BUSINESS_ANALYST/Business Analyst course/Econometrics and Causal Inference/Google Causal Impact


In [ ]:
# Install Libraries
!pip install yfinance
!pip install tfcausalimpact

In [ ]:
# Import Libraries
import yfinance as yf
from causalimpact import CausalImpact
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [ ]:
# Define Dates
training_start = "2020-09-01"
training_end = "2020-10-19"
treatment_start = "2020-10-20"
treatment_end = "2020-10-23"
end_stock = "2020-10-24"

# Loading Financial Data

In [ ]:
# Bitcoin Data
BitC = ['BTC-USD']
y = yf.download(tickers=BitC, 
                start = training_start, 
                end = end_stock,
                interval = '1d')
y = y['Adj Close'].rename('y')
y.head(2)

[*********************100%***********************]  1 of 1 completed


Date
2020-09-01    11970.478516
2020-09-02    11414.034180
Name: y, dtype: float64

In [ ]:
# Load more stock data (of control groups)
stocks = ["CARL-B.CO", "ZAL.DE", "SQ", "CRSP", "TRMB", "JD", "DE", "KTOS", "GOOG"]
X = yf.download(tickers = stocks, 
                start = training_start, 
                end = end_stock,
                interval = "1d")
X.head(3)


[*********************100%***********************]  9 of 9 completed


Adj Close                                                 \
             CARL-B.CO       CRSP          DE         GOOG         JD   
Date                                                                    
2020-09-01  816.406250  93.419998  213.404526  1660.709961  80.459969   
2020-09-02  824.950073  93.930000  212.835938  1728.280029  81.162254   
2020-09-03  840.708557  85.690002  206.659958  1641.839966  77.094872   

                                                              Close  ...  \
                 KTOS          SQ       TRMB     ZAL.DE   CARL-B.CO  ...   
Date                                                                 ...   
2020-09-01  19.700001  166.660004  53.410000  77.000000  860.000000  ...   
2020-09-02  19.910000  162.880005  54.310001  77.000000  869.000000  ...   
2020-09-03  19.520000  152.860001  50.900002  75.080002  885.599976  ...   

                 Open    Volume                                               \
               ZAL.DE CARL-B.CO       CRSP         DE       GOOG          JD   
Date                                                                           
2020-09-01  74.959999    269532   779500.0  1873400.0  1825300.0  11431400.0   
2020-09-02  77.500000    202539   532000.0  1983000.0  2511200.0  13860900.0   
2020-09-03  77.260002    248867  1278900.0  1975200.0  3107800.0  19254000.0   

                                                     
                KTOS          SQ       TRMB  ZAL.DE  
Date                                                 
2020-09-01  587600.0  12306400.0   542300.0  790198  
2020-09-02  612800.0  11214800.0   728800.0  496554  
2020-09-03  847100.0  16421200.0  1220300.0  736259  

[3 rows x 54 columns]

In [ ]:
# Getting just the adjusted close 
X = X.iloc[:,:len(stocks)]
X.head(2)

Adj Close                                                 \
             CARL-B.CO       CRSP          DE         GOOG         JD   
Date                                                                    
2020-09-01  816.406250  93.419998  213.404526  1660.709961  80.459969   
2020-09-02  824.950073  93.930000  212.835938  1728.280029  81.162254   

                                                     
                 KTOS          SQ       TRMB ZAL.DE  
Date                                                 
2020-09-01  19.700001  166.660004  53.410000   77.0  
2020-09-02  19.910000  162.880005  54.310001   77.0

In [ ]:
# Removing unneded level
X.columns = X.columns.droplevel()
X.head(2)

,CARL-B.CO,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE
Date,,,,,,,,,
2020-09-01,816.406250,93.419998,213.404526,1660.709961,80.459969,19.700001,166.660004,53.410000,77.0
2020-09-02,824.950073,93.930000,212.835938,1728.280029,81.162254,19.910000,162.880005,54.310001,77.0


In [ ]:
# Combine everything
df = pd.concat([y,X], axis = 1)
df.head(2)

,y,CARL-B.CO,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE
Date,,,,,,,,,,
2020-09-01,11970.478516,816.406250,93.419998,213.404526,1660.709961,80.459969,19.700001,166.660004,53.410000,77.0
2020-09-02,11414.034180,824.950073,93.930000,212.835938,1728.280029,81.162254,19.910000,162.880005,54.310001,77.0
